In [5]:
# lead lag pearson correlation, negative values indicate data1 leads data2, n is the lead lag step
# the time length for correlation is sample-n; for example, for 50 years sample,
# if 10 yrs leadlag is choosen, the time length is 40 yrs, if lagyear =7, it will be maxmum lead year 7 and lag year 7

import pandas as pd
import xarray as xr
import numpy as np

na = np.load("/work/uo1075/u241321/data/eemd_NAO_DJF.npy")   # linealy detrened for each grid point  
nao = na[2,:]


T = np.load("/work/uo1075/u241321/data/eemd_t310_assi_Norwegian.npy")[2,:]
rawcorr      = np.load("/work/uo1075/u241321/data/leadlag_NAO_T_c2.npy")




In [6]:
# lead-lag correlation
from scipy.stats import pearsonr
import numpy as np

# lead lag pearson correlation, negative values indicate data1 leads data2, n is the lead lag step
# the time length for correlation is sample-n; for example, for 50 years sample,
# if 10 yrs leadlag is choosen, the time length is 40 yrs

def leadlagcor(data1,data2,n):
    a=-n
    b=-a
    c=b*2+1
    x=np.arange(-n,n+1,1)
    r=np.zeros((c,1))
    p=np.zeros((c,1))
    begin=x[c-1]
    end=len(data1)-x[c-1]
    
    for i in range(c):
        if i<=(b):
            r[n-i],p[n-i]=pearsonr(data1[:end], data2[i:end+i])
        else:
            r[i],p[i]=pearsonr(data1[x[i]:end+x[i]], data2[:end])
    return r

In [7]:
# resampled_coef_data = []
# # for sample in data:
# resample = _bootstrap_resample(nao, T, nlagyear, block_length=block_length)
# resampled_coef_data.append(resample)
y1 = nao
y2 = T
nlagyear = 7
c=2*nlagyear+1
sig=np.zeros((2*nlagyear+1))
block_length=2
n_resamples=1000
confidence_level=0.95

for j in range(c):
    
    # bootstrap

    def _bootstrap_resample(sample1, sample2, nlagyear, block_length, n_resamples=None, random_state=None):
        """Bootstrap resample the sample."""
   
        np.random.shuffle(sample1.reshape((-1,block_length)))
        rng_state = np.random.get_state()
        np.random.set_state(rng_state)
        np.random.shuffle(sample2.reshape((-1,block_length)))
    
    # resamples = sample
        corr = leadlagcor(sample1, sample2, nlagyear)[j]
    # bootstrap - each row is a random resample of original observations
    # i = rng_integers(random_state, 0, n, (n_resamples, n))

    # resamples = sample[..., i]
        return corr
    theta_hat_b = []
    for k in range(0, n_resamples):
        # batch_actual = min(batch_nominal, n_resamples-k)
        # Generate resamples
        # theta_hat_b = [] 
        resampled_coef_data = []
        # for sample in data:
        resample = _bootstrap_resample(y1, y2, nlagyear, block_length)
        resampled_coef_data.append(resample)
        theta_hat_b.append(resampled_coef_data)
    theta_hat_b = np.concatenate(theta_hat_b, axis=-1)
        # Calculate percentile interval
    alpha = (1 - confidence_level)/2
    
    interval = alpha, 1-alpha

    def percentile_fun(a, q):
        return np.percentile(a=a, q=q, axis=-1)

    # Calculate confidence interval of statistic
    ci_l = percentile_fun(theta_hat_b, interval[0]*100)
    ci_u = percentile_fun(theta_hat_b, interval[1]*100)

    if rawcorr[j] >= ci_u:
        sig[j] = 1
    else:
        if rawcorr[j] <= ci_l:
            sig[j] = 1
        else:
            sig[j] = 0
        
sig

array([0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0.])

In [8]:
rawcorr

array([[-0.12865955],
       [-0.42084872],
       [-0.43917944],
       [-0.34373878],
       [-0.18391949],
       [ 0.12614619],
       [ 0.47049664],
       [ 0.52451693],
       [ 0.19367211],
       [-0.17615581],
       [-0.3192819 ],
       [-0.29121874],
       [-0.21613759],
       [-0.06584287],
       [ 0.180993  ]])